# Persiapan Library

In [1]:
%pip install -q pyspellchecker
%pip install -q Sastrawi

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# library awal untuk perhitungan dan pengolahan teks
import numpy as np
import re
import pandas as pd
import pickle

from collections import Counter

# monitoring
from tqdm import tqdm

# library untuk praproses teks
import nltk

# 1. Stop Word
nltk.download('stopwords')
from nltk.corpus import stopwords

# 3. Cek Ejaan Pembakuan kata B.inggris
from spellchecker import SpellChecker
# Inisialisasi SpellChecker untuk bahasa Indonesia
spell = SpellChecker(language='en')

# 4.Stemming
## Indo
nltk.download('wordnet')
# Menggunaakn Sastrawi untuk memproses teks dalam bahasa Indonesia.
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()

## English
from nltk.stem import PorterStemmer, WordNetLemmatizer
stemmer = PorterStemmer()

# 5. Tokenisasi
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\willy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\willy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\willy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

## Praproses Teks

## Membuat Fungsi

### Menghilangkan symbol atau tanda baca

In [3]:
# Case Folding
def clean_lower(lwr):
    lwr = lwr.lower() # lowercase text
    return lwr

# Menghapus tanda baca, angka, dan simbol
def clean_punct(text):
    clean_spcl = re.compile('[/(){}\[\]\|@,;_]')
    clean_symbol = re.compile('[^0-9a-z]')
    clean_number = re.compile('[0-9]')
    text = clean_spcl.sub('', text)
    text = clean_symbol.sub(' ', text)
    text = clean_number.sub('', text)
    return text

# Menghaps double atau lebih whitespace
def _normalize_whitespace(text):
    corrected = str(text)
    corrected = re.sub(r"//t",r"\t", corrected)
    corrected = re.sub(r"( )\1+",r"\1", corrected)
    corrected = re.sub(r"(\n)\1+",r"\1", corrected)
    corrected = re.sub(r"(\r)\1+",r"\1", corrected)
    corrected = re.sub(r"(\t)\1+",r"\1", corrected)
    return corrected.strip(" ")

# Menghapus stopwords Indonesia
def clean_stopwords_id(text):
    stopword = set(stopwords.words('indonesian'))
    text = ' '.join(word for word in text.split() if word not in stopword) # hapus stopword dari kolom deskripsi
    return text

# Menghapus stopwords Inggris
def clean_stopwords_en(text):
    stopword = set(stopwords.words('english'))
    text = ' '.join(word for word in text.split() if word not in stopword) # hapus stopword dari kolom deskripsi
    return text

# Stemming with Sastrawi
def sastrawistemmer(text):
    st = factory.create_stemmer()
    text = ' '.join(st.stem(word) for word in tqdm(text.split()) if word in text)
    return text

# Stemming with Porter Stemmer
def porter_stemming(text):
    # Memecah teks menjadi kata-kata
    words = nltk.word_tokenize(text)  # Tokenisasi
    # Melakukan stemming pada setiap kata
    stemmed_words = [stemmer.stem(word) for word in tqdm(words)]
    # Menggabungkan kata-kata yang telah di-stem
    stemmed_text = ' '.join(stemmed_words)
    return stemmed_text

# Fungsi untuk memperbaiki ejaan dalam bahasa Inggris dengan tqdm
def correct_spelling_english(text):
    if not isinstance(text, str):
        return text  # Mengabaikan nilai non-string
    words = text.split()
    corrected_words = []
    # Menggunakan tqdm untuk menampilkan progres perbaikan ejaan
    for word in tqdm(words, desc="Fix Spell", unit="kata"):
        corrected = spell.correction(word)  # Periksa ejaan kata
        # Jika spell.correction mengembalikan None, gunakan kata aslinya atau string kosong
        if corrected is None:
            corrected_words.append(word)
        else:
            corrected_words.append(corrected)

    corrected_text = ' '.join(corrected_words)
    return corrected_text

# Fungsi untuk melakukan tokenisasi pada teks
def tokenize_text(text):
    if isinstance(text, str):  # Memastikan input adalah string
        return word_tokenize(text)  # Mengembalikan token
    return []  # Kembali ke list kosong jika bukan string


## PTA Berita Online Kompas.com

### Load Data

#### Data PTA

In [4]:
main_df_berita = pd.read_csv('datasets/berita.csv', delimiter=',')
main_df_berita.head()

,No,judul,berita,tanggal,kategori,link
0,1,Airlangga Harap Kenaikan UMP Tingkatkan Daya B...,Menteri Koordinator (Menko) Bidang Perekonomia...,"Minggu, 01 Des 2024 23:40 WIB",Ekonomi,https://www.cnnindonesia.com/ekonomi/202412012...
1,2,PT SIER Beri Penghargaan untuk 50 Tenant Terba...,"Dalam rangka memeriahkan hari jadi ke-50, PT S...","Minggu, 01 Des 2024 20:45 WIB",Ekonomi,https://www.cnnindonesia.com/ekonomi/202412012...
2,3,Prabowo Bakal Bentuk Kementerian Penerimaan Ne...,Wacana Presiden Prabowo Subianto akan membentu...,"Minggu, 01 Des 2024 19:40 WIB",Ekonomi,https://www.cnnindonesia.com/ekonomi/202412011...
3,4,Sinergi Kemenag & BPJS Ketenagakerjaan Lindung...,BPJS Ketenagakerjaan dan Kementerian Agama (Ke...,"Minggu, 01 Des 2024 19:03 WIB",Ekonomi,https://www.cnnindonesia.com/ekonomi/202412011...
4,5,Pemerintah Segera Bentuk Satgas PHK Usai Tetap...,Pemerintah akan segera membentuk Satuan Tugas ...,"Minggu, 01 Des 2024 19:00 WIB",Ekonomi,https://www.cnnindonesia.com/ekonomi/202412011...


In [5]:
main_df_berita['kategori'].value_counts()

kategori
Ekonomi          375
Olahraga         375
Nasional         375
Internasional    375
Name: count, dtype: int64

#### Remove data Missing

### No Stop Word and Stemming

In [ ]:
main_df_berita = main_df_berita[["berita"]].dropna()
main_df_berita.isnull().sum()

berita    0
dtype: int64

#### Lower case

In [11]:
main_df_berita['lwr_indo'] = main_df_berita['berita'].apply(clean_lower)
casefolding=pd.DataFrame(main_df_berita[["berita","lwr_indo"]])
casefolding

,berita,lwr_indo
0,Menteri Koordinator (Menko) Bidang Perekonomia...,menteri koordinator (menko) bidang perekonomia...
1,"Dalam rangka memeriahkan hari jadi ke-50, PT S...","dalam rangka memeriahkan hari jadi ke-50, pt s..."
2,Wacana Presiden Prabowo Subianto akan membentu...,wacana presiden prabowo subianto akan membentu...
3,BPJS Ketenagakerjaan dan Kementerian Agama (Ke...,bpjs ketenagakerjaan dan kementerian agama (ke...
4,Pemerintah akan segera membentuk Satuan Tugas ...,pemerintah akan segera membentuk satuan tugas ...
...,...,...
1495,Anggota Dewan Keselamatan Transportasi Nasiona...,anggota dewan keselamatan transportasi nasiona...
1496,Israel telah membebaskan 110 tahanan Palestina...,israel telah membebaskan 110 tahanan palestina...
1497,Hamas mengonfirmasi kematian kepala militernya...,hamas mengonfirmasi kematian kepala militernya...
1498,Tim penyelam diduga menemukan satu dari dua bl...,tim penyelam diduga menemukan satu dari dua bl...


#### Menghilangkan symbol atau tanda baca

In [12]:
main_df_berita['clean_stb_indo'] = main_df_berita['lwr_indo'].apply(clean_punct)
clean_punct_df = pd.DataFrame(main_df_berita[['lwr_indo','clean_stb_indo']])
clean_punct_df

,lwr_indo,clean_stb_indo
0,menteri koordinator (menko) bidang perekonomia...,menteri koordinator menko bidang perekonomian ...
1,"dalam rangka memeriahkan hari jadi ke-50, pt s...",dalam rangka memeriahkan hari jadi ke pt sura...
2,wacana presiden prabowo subianto akan membentu...,wacana presiden prabowo subianto akan membentu...
3,bpjs ketenagakerjaan dan kementerian agama (ke...,bpjs ketenagakerjaan dan kementerian agama kem...
4,pemerintah akan segera membentuk satuan tugas ...,pemerintah akan segera membentuk satuan tugas ...
...,...,...
1495,anggota dewan keselamatan transportasi nasiona...,anggota dewan keselamatan transportasi nasiona...
1496,israel telah membebaskan 110 tahanan palestina...,israel telah membebaskan tahanan palestina pa...
1497,hamas mengonfirmasi kematian kepala militernya...,hamas mengonfirmasi kematian kepala militernya...
1498,tim penyelam diduga menemukan satu dari dua bl...,tim penyelam diduga menemukan satu dari dua bl...


##### Cek Ejaan Peter Norvig

###### Fungsi

In [13]:
# Spell checker (default bahasa Inggris, kita isi dengan kosakata dataset CNN)
spell = SpellChecker(language=None)
unique_words = set(" ".join(main_df_berita["clean_stb_indo"].astype(str)).split())
print(unique_words)
spell.word_frequency.load_words(unique_words)

def spellcheck_text(text):
    if pd.isna(text):
        return ""
    tokens = text.split()
    corrected_tokens = []
    for token in tokens:
        if token not in spell:
            correction = spell.correction(token)
            corrected_tokens.append(correction if correction else token)
        else:
            corrected_tokens.append(token)
    return " ".join(corrected_tokens)

{'utk', 'membiayai', 'sarung', 'redaksikronologi', 'malaysiahasilnya', 'yaituganda', 'rahang', 'terdengar', 'pot', 'roslan', 'natakusumah', 'masoud', 'riza', 'alliya', 'dokumensementara', 'memperlakukan', 'popularitas', 'disemprot', 'nze', 'soebianto', 'oleng', 'mandalika', 'berkaca', 'pemecahan', 'newsbudi', 'terlambat', 'wuarbanaran', 'mandatorinya', 'cekal', 'sanggahpada', 'febrana', 'declan', 'tertutupmeski', 'perizinan', 'espoir', 'kenly', 'eksel', 'jagakarsa', 'sparks', 'kasatreskrim', 'kasad', 'shinawatra', 'gwijangge', 'banjar', 'bermesin', 'memangkas', 'menepi', 'vitidsarn', 'shu', 'satya', 'dipertajam', 'menyejahterakan', 'yuliot', 'pemukim', 'idle', 'penuh', 'providernya', 'asasi', 'balap', 'merekayasa', 'arifah', 'nitro', 'istana', 'panjat', 'sabater', 'denmarkhungariarepublik', 'theatre', 'blora', 'dipakai', 'zola', 'masalah', 'motogp', 'disuguhkan', 'pacuan', 'artifisial', 'rizal', 'uma', 'rizkynata', 'iwga', 'autp', 'pendukungnya', 'biji', 'korupsi', 'indonesiamemasuki',

#### Cek Ejaan Pembakuan Kata

In [32]:
# main_df_berita = main_df_berita.head(10)

In [14]:
main_df_berita['clean_typo_indo'] = main_df_berita['clean_stb_indo'].apply(spellcheck_text)
clean_spell=pd.DataFrame(main_df_berita[["clean_stb_indo",'clean_typo_indo']])
clean_spell

,clean_stb_indo,clean_typo_indo
0,menteri koordinator menko bidang perekonomian ...,menteri koordinator menko bidang perekonomian ...
1,dalam rangka memeriahkan hari jadi ke pt sura...,dalam rangka memeriahkan hari jadi ke pt surab...
2,wacana presiden prabowo subianto akan membentu...,wacana presiden prabowo subianto akan membentu...
3,bpjs ketenagakerjaan dan kementerian agama kem...,bpjs ketenagakerjaan dan kementerian agama kem...
4,pemerintah akan segera membentuk satuan tugas ...,pemerintah akan segera membentuk satuan tugas ...
...,...,...
1495,anggota dewan keselamatan transportasi nasiona...,anggota dewan keselamatan transportasi nasiona...
1496,israel telah membebaskan tahanan palestina pa...,israel telah membebaskan tahanan palestina pad...
1497,hamas mengonfirmasi kematian kepala militernya...,hamas mengonfirmasi kematian kepala militernya...
1498,tim penyelam diduga menemukan satu dari dua bl...,tim penyelam diduga menemukan satu dari dua bl...


In [13]:
main_df_berita['tokenize_indo'] = main_df_berita['clean_typo_indo'].apply(tokenize_text)
tokenize=pd.DataFrame(main_df_berita[['clean_typo_indo','tokenize_indo']])
tokenize

,clean_typo_indo,tokenize_indo
0,kompas com menteri pemuda dan olahraga erick t...,"[kompas, com, menteri, pemuda, dan, olahraga, ..."
1,kompas com manajer chelsea enzo maresca mengan...,"[kompas, com, manajer, chelsea, enzo, maresca,..."
2,kompas com pelatih liverpool arne slot mengaku...,"[kompas, com, pelatih, liverpool, arne, slot, ..."
3,kompas com hasil terbaru pekan kelima liga ita...,"[kompas, com, hasil, terbaru, pekan, kelima, l..."
4,kompas com menteri pemuda dan olahraga republi...,"[kompas, com, menteri, pemuda, dan, olahraga, ..."
...,...,...
1195,karanganyar kompas com dapur satuan pelayanan ...,"[karanganyar, kompas, com, dapur, satuan, pela..."
1196,bandung kompas com kepala stasiun geofisika bm...,"[bandung, kompas, com, kepala, stasiun, geofis..."
1197,tegal kompas com maraknya kasus keracunan maka...,"[tegal, kompas, com, maraknya, kasus, keracuna..."
1198,solo kompas com pemerintah kota pemkot solo ja...,"[solo, kompas, com, pemerintah, kota, pemkot, ..."


In [3]:
import numpy as np

In [ ]:
df_berita = df_final_sample['kategori']
df = main_df_berita

# Gabungkan kedua DataFrame secara horizontal
df_gabungan = pd.concat([df, df_berita], axis=1)
df_gabungan.to_csv('datasets/df_preprocessing_no_stemming.csv', index=False)

### Full Processing

In [ ]:
main_df_berita = df_final_sample[["Isi Berita"]].dropna()
main_df_berita.isnull().sum()
main_df_berita.head()

,Isi Berita
0,KOMPAS.com - Menteri Pemuda dan Olahraga Erick...
1,"KOMPAS.com - Manajer Chelsea, Enzo Maresca men..."
2,"KOMPAS.com - Pelatih Liverpool, Arne Slot, men..."
3,KOMPAS.com - Hasil terbaru pekan kelima Liga I...
4,KOMPAS.com - Menteri Pemuda dan Olahraga Repub...


#### Lower case

In [ ]:
main_df_berita['lwr_indo'] = main_df_berita['Isi Berita'].apply(clean_lower)
casefolding=pd.DataFrame(main_df_berita[["Isi Berita","lwr_indo"]])
casefolding

,Isi Berita,lwr_indo
0,KOMPAS.com - Menteri Pemuda dan Olahraga Erick...,kompas.com - menteri pemuda dan olahraga erick...
1,"KOMPAS.com - Manajer Chelsea, Enzo Maresca men...","kompas.com - manajer chelsea, enzo maresca men..."
2,"KOMPAS.com - Pelatih Liverpool, Arne Slot, men...","kompas.com - pelatih liverpool, arne slot, men..."
3,KOMPAS.com - Hasil terbaru pekan kelima Liga I...,kompas.com - hasil terbaru pekan kelima liga i...
4,KOMPAS.com - Menteri Pemuda dan Olahraga Repub...,kompas.com - menteri pemuda dan olahraga repub...
...,...,...
1195,"KARANGANYAR, KOMPAS.com - Dapur Satuan Pelayan...","karanganyar, kompas.com - dapur satuan pelayan..."
1196,"BANDUNG, KOMPAS.com -Kepala Stasiun Geofisika ...","bandung, kompas.com -kepala stasiun geofisika ..."
1197,"TEGAL, KOMPAS.com – Maraknya kasus keracunan m...","tegal, kompas.com – maraknya kasus keracunan m..."
1198,"SOLO, KOMPAS.com - Pemerintah Kota (Pemkot) So...","solo, kompas.com - pemerintah kota (pemkot) so..."


#### Clean Stopwords

In [ ]:
# Buat kolom tambahan untuk data description yang telah dilakukan proses penghapusan stopwords
main_df_berita['clean_sw_indo'] = main_df_berita['lwr_indo'].apply(clean_stopwords_id)
stop_words = pd.DataFrame(main_df_berita[['lwr_indo','clean_sw_indo']])
stop_words

,lwr_indo,clean_sw_indo
0,kompas.com - menteri pemuda dan olahraga erick...,kompas.com - menteri pemuda olahraga erick tho...
1,"kompas.com - manajer chelsea, enzo maresca men...","kompas.com - manajer chelsea, enzo maresca men..."
2,"kompas.com - pelatih liverpool, arne slot, men...","kompas.com - pelatih liverpool, arne slot, men..."
3,kompas.com - hasil terbaru pekan kelima liga i...,kompas.com - hasil terbaru pekan liga italia 2...
4,kompas.com - menteri pemuda dan olahraga repub...,kompas.com - menteri pemuda olahraga republik ...
...,...,...
1195,"karanganyar, kompas.com - dapur satuan pelayan...","karanganyar, kompas.com - dapur satuan pelayan..."
1196,"bandung, kompas.com -kepala stasiun geofisika ...","bandung, kompas.com -kepala stasiun geofisika ..."
1197,"tegal, kompas.com – maraknya kasus keracunan m...","tegal, kompas.com – maraknya keracunan makanan..."
1198,"solo, kompas.com - pemerintah kota (pemkot) so...","solo, kompas.com - pemerintah kota (pemkot) so..."


#### Menghilangkan symbol atau tanda baca

In [ ]:
main_df_berita['clean_stb_indo'] = main_df_berita['clean_sw_indo'].apply(clean_punct)
clean_punct_df = pd.DataFrame(main_df_berita[['clean_sw_indo','clean_stb_indo']])
clean_punct_df

,clean_sw_indo,clean_stb_indo
0,kompas.com - menteri pemuda olahraga erick tho...,kompas com menteri pemuda olahraga erick tho...
1,"kompas.com - manajer chelsea, enzo maresca men...",kompas com manajer chelsea enzo maresca meng...
2,"kompas.com - pelatih liverpool, arne slot, men...",kompas com pelatih liverpool arne slot menga...
3,kompas.com - hasil terbaru pekan liga italia 2...,kompas com hasil terbaru pekan liga italia ...
4,kompas.com - menteri pemuda olahraga republik ...,kompas com menteri pemuda olahraga republik ...
...,...,...
1195,"karanganyar, kompas.com - dapur satuan pelayan...",karanganyar kompas com dapur satuan pelayana...
1196,"bandung, kompas.com -kepala stasiun geofisika ...",bandung kompas com kepala stasiun geofisika b...
1197,"tegal, kompas.com – maraknya keracunan makanan...",tegal kompas com maraknya keracunan makanan ...
1198,"solo, kompas.com - pemerintah kota (pemkot) so...",solo kompas com pemerintah kota pemkot solo ...


In [ ]:
main_df_berita['clean_stb_indo'] = main_df_berita['lwr_indo'].apply(clean_punct)
clean_punct_df = pd.DataFrame(main_df_berita[['lwr_indo','clean_stb_indo']])
clean_punct_df

##### Cek Ejaan Peter Norvig

###### Fungsi

In [ ]:
# Spell checker (default bahasa Inggris, kita isi dengan kosakata dataset CNN)
spell = SpellChecker(language=None)
unique_words = set(" ".join(main_df_berita["clean_stb_indo"].astype(str)).split())
print(unique_words)
spell.word_frequency.load_words(unique_words)

def spellcheck_text(text):
    if pd.isna(text):
        return ""
    tokens = text.split()
    corrected_tokens = []
    for token in tokens:
        if token not in spell:
            correction = spell.correction(token)
            corrected_tokens.append(correction if correction else token)
        else:
            corrected_tokens.append(token)
    return " ".join(corrected_tokens)

{'gozali', 'nantinya', 'ternama', 'setinggi', 'simanjuntak', 'menunggu', 'mengupayakan', 'pecal', 'ffws', 'bournemouth', 'guncang', 'randy', 'gemuruh', 'syukurnya', 'ditargetkan', 'penghujungnya', 'pertamaxmenurutnya', 'tidur', 'baiklah', 'menghanguskan', 'rafni', 'wamenkeu', 'diblokir', 'dicat', 'comharyanti', 'alami', 'landscape', 'melakukannya', 'dsp', 'gratisan', 'psikis', 'kerinduannya', 'mobil', 'humas', 'lansiran', 'bachtiar', 'menyelesaikannya', 'ratas', 'peltu', 'maskulin', 'melaksanakannya', 'kombes', 'mengunduh', 'rambut', 'coupe', 'bapakibu', 'jammalik', 'masif', 'akur', 'balangan', 'tambahkan', 'deja', 'keliling', 'roadside', 'lebarnya', 'saudaranya', 'uncovered', 'peleknya', 'scoopydampaknya', 'awreceh', 'bertuliskan', 'dipertahankan', 'gatot', 'circular', 'marathon', 'responsibility', 'kerajinan', 'jejeran', 'rejoagung', 'memantau', 'casc', 'note', 'mosquera', 'budiono', 'mild', 'pelalawan', 'merapikan', 'sedih', 'irakselain', 'teken', 'aren', 'josua', 'sawah', 'sirenese

#### Cek Ejaan Pembakuan Kata

In [ ]:
# main_df_berita = main_df_berita.head(10)

In [ ]:
main_df_berita['clean_typo_indo'] = main_df_berita['clean_stb_indo'].apply(spellcheck_text)
clean_spell=pd.DataFrame(main_df_berita[["clean_stb_indo",'clean_typo_indo']])
clean_spell

,clean_stb_indo,clean_typo_indo
0,kompas com menteri pemuda olahraga erick tho...,kompas com menteri pemuda olahraga erick thohi...
1,kompas com manajer chelsea enzo maresca meng...,kompas com manajer chelsea enzo maresca mengan...
2,kompas com pelatih liverpool arne slot menga...,kompas com pelatih liverpool arne slot mengaku...
3,kompas com hasil terbaru pekan liga italia ...,kompas com hasil terbaru pekan liga italia per...
4,kompas com menteri pemuda olahraga republik ...,kompas com menteri pemuda olahraga republik in...
...,...,...
1195,karanganyar kompas com dapur satuan pelayana...,karanganyar kompas com dapur satuan pelayanan ...
1196,bandung kompas com kepala stasiun geofisika b...,bandung kompas com kepala stasiun geofisika bm...
1197,tegal kompas com maraknya keracunan makanan ...,tegal kompas com maraknya keracunan makanan pr...
1198,solo kompas com pemerintah kota pemkot solo ...,solo kompas com pemerintah kota pemkot solo ja...


In [ ]:
main_df_berita['stemming_indo'] = main_df_berita['clean_typo_indo'].apply(sastrawistemmer)
stemming=pd.DataFrame(main_df_berita[['clean_typo_indo','stemming_indo']])
stemming

100%|██████████| 235/235 [00:05<00:00, 43.69it/s]


,clean_typo_indo,stemming_indo
0,kompas com menteri pemuda olahraga erick thohi...,kompas com menteri pemuda olahraga erick thohi...
1,kompas com manajer chelsea enzo maresca mengan...,kompas com manajer chelsea enzo maresca anggap...
2,kompas com pelatih liverpool arne slot mengaku...,kompas com latih liverpool arne slot aku cryst...
3,kompas com hasil terbaru pekan liga italia per...,kompas com hasil baru pekan liga italia beda n...
4,kompas com menteri pemuda olahraga republik in...,kompas com menteri pemuda olahraga republik in...
...,...,...
1195,karanganyar kompas com dapur satuan pelayanan ...,karanganyar kompas com dapur satu layan penuh ...
1196,bandung kompas com kepala stasiun geofisika bm...,bandung kompas com kepala stasiun geofisika bm...
1197,tegal kompas com maraknya keracunan makanan pr...,tegal kompas com marak racun makan program mak...
1198,solo kompas com pemerintah kota pemkot solo ja...,solo kompas com perintah kota pemkot solo jawa...


In [ ]:
main_df_berita['tokenize_indo'] = main_df_berita['stemming_indo'].apply(tokenize_text)
tokenize=pd.DataFrame(main_df_berita[['stemming_indo','tokenize_indo']])
tokenize

,stemming_indo,tokenize_indo
0,kompas com menteri pemuda olahraga erick thohi...,"[kompas, com, menteri, pemuda, olahraga, erick..."
1,kompas com manajer chelsea enzo maresca anggap...,"[kompas, com, manajer, chelsea, enzo, maresca,..."
2,kompas com latih liverpool arne slot aku cryst...,"[kompas, com, latih, liverpool, arne, slot, ak..."
3,kompas com hasil baru pekan liga italia beda n...,"[kompas, com, hasil, baru, pekan, liga, italia..."
4,kompas com menteri pemuda olahraga republik in...,"[kompas, com, menteri, pemuda, olahraga, repub..."
...,...,...
1195,karanganyar kompas com dapur satu layan penuh ...,"[karanganyar, kompas, com, dapur, satu, layan,..."
1196,bandung kompas com kepala stasiun geofisika bm...,"[bandung, kompas, com, kepala, stasiun, geofis..."
1197,tegal kompas com marak racun makan program mak...,"[tegal, kompas, com, marak, racun, makan, prog..."
1198,solo kompas com perintah kota pemkot solo jawa...,"[solo, kompas, com, perintah, kota, pemkot, so..."


In [ ]:
# Simpan DataFrame ke dalam file CSV
# main_df_berita.to_csv('datasets/df_berita_preprocessing.csv', index=False)

In [ ]:
main_df_berita

,Isi Berita,lwr_indo,clean_sw_indo,clean_stb_indo,clean_typo_indo,stemming_indo,tokenize_indo
0,KOMPAS.com - Menteri Pemuda dan Olahraga Erick...,kompas.com - menteri pemuda dan olahraga erick...,kompas.com - menteri pemuda olahraga erick tho...,kompas com menteri pemuda olahraga erick tho...,kompas com menteri pemuda olahraga erick thohi...,kompas com menteri pemuda olahraga erick thohi...,"[kompas, com, menteri, pemuda, olahraga, erick..."
1,"KOMPAS.com - Manajer Chelsea, Enzo Maresca men...","kompas.com - manajer chelsea, enzo maresca men...","kompas.com - manajer chelsea, enzo maresca men...",kompas com manajer chelsea enzo maresca meng...,kompas com manajer chelsea enzo maresca mengan...,kompas com manajer chelsea enzo maresca anggap...,"[kompas, com, manajer, chelsea, enzo, maresca,..."
2,"KOMPAS.com - Pelatih Liverpool, Arne Slot, men...","kompas.com - pelatih liverpool, arne slot, men...","kompas.com - pelatih liverpool, arne slot, men...",kompas com pelatih liverpool arne slot menga...,kompas com pelatih liverpool arne slot mengaku...,kompas com latih liverpool arne slot aku cryst...,"[kompas, com, latih, liverpool, arne, slot, ak..."
3,KOMPAS.com - Hasil terbaru pekan kelima Liga I...,kompas.com - hasil terbaru pekan kelima liga i...,kompas.com - hasil terbaru pekan liga italia 2...,kompas com hasil terbaru pekan liga italia ...,kompas com hasil terbaru pekan liga italia per...,kompas com hasil baru pekan liga italia beda n...,"[kompas, com, hasil, baru, pekan, liga, italia..."
4,KOMPAS.com - Menteri Pemuda dan Olahraga Repub...,kompas.com - menteri pemuda dan olahraga repub...,kompas.com - menteri pemuda olahraga republik ...,kompas com menteri pemuda olahraga republik ...,kompas com menteri pemuda olahraga republik in...,kompas com menteri pemuda olahraga republik in...,"[kompas, com, menteri, pemuda, olahraga, repub..."
...,...,...,...,...,...,...,...
1195,"KARANGANYAR, KOMPAS.com - Dapur Satuan Pelayan...","karanganyar, kompas.com - dapur satuan pelayan...","karanganyar, kompas.com - dapur satuan pelayan...",karanganyar kompas com dapur satuan pelayana...,karanganyar kompas com dapur satuan pelayanan ...,karanganyar kompas com dapur satu layan penuh ...,"[karanganyar, kompas, com, dapur, satu, layan,..."
1196,"BANDUNG, KOMPAS.com -Kepala Stasiun Geofisika ...","bandung, kompas.com -kepala stasiun geofisika ...","bandung, kompas.com -kepala stasiun geofisika ...",bandung kompas com kepala stasiun geofisika b...,bandung kompas com kepala stasiun geofisika bm...,bandung kompas com kepala stasiun geofisika bm...,"[bandung, kompas, com, kepala, stasiun, geofis..."
1197,"TEGAL, KOMPAS.com – Maraknya kasus keracunan m...","tegal, kompas.com – maraknya kasus keracunan m...","tegal, kompas.com – maraknya keracunan makanan...",tegal kompas com maraknya keracunan makanan ...,tegal kompas com maraknya keracunan makanan pr...,tegal kompas com marak racun makan program mak...,"[tegal, kompas, com, marak, racun, makan, prog..."
1198,"SOLO, KOMPAS.com - Pemerintah Kota (Pemkot) So...","solo, kompas.com - pemerintah kota (pemkot) so...","solo, kompas.com - pemerintah kota (pemkot) so...",solo kompas com pemerintah kota pemkot solo ...,solo kompas com pemerintah kota pemkot solo ja...,solo kompas com perintah kota pemkot solo jawa...,"[solo, kompas, com, perintah, kota, pemkot, so..."


In [ ]:
# df_final_sample.to_csv('datasets/df_berita_200.csv', index=False)

In [ ]:
# import numpy as np

In [ ]:
# df_berita = pd.read_csv("datasets/df_berita_200.csv", usecols=['Kategori Berita'])
# df = pd.read_csv("datasets/df_berita_preprocessing.csv")

# # Gabungkan kedua DataFrame secara horizontal
# df_gabungan = pd.concat([df, df_berita], axis=1)
# df_gabungan.to_csv('datasets/df_preprocessing.csv.csv', index=False)

In [ ]:
# df = pd.read_csv("datasets/df_berita_preprocessing.csv")
# df = pd.read_csv("datasets/df_berita_preprocessing.csv")